# Whatsapp Web Automation

**Author: Jimmy Lam**

**_Created: 2022-04-25_**

v1.0: Public Release

**Part 1 - Import libraries and launch Whatsapp Web**

In [21]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FirefoxOptions
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import time
import logging
import json
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException


logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

# chrome_options = Options()
# chromedriver_path = '/Users/rajath/chromedriver-mac-arm64/chromedriver'
# service = Service(executable_path=chromedriver_path)
# driver = webdriver.Chrome(service=service, options=chrome_options)

firefox_options = FirefoxOptions()
geckodriver_path = 'C:\\Users\\Administrator\\Documents\\geckodriver.exe'  # Update this path
service = FirefoxService(executable_path=geckodriver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.get('https://web.whatsapp.com/')

#wait 60 secs to allow for the user to manually scan the Whatsapp Web QR code to log on
el_side = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, "side")))

#locate the search box
# el_search = el_side.find_element(By.XPATH, "//div[contains(@title, 'Search')]")
# print("Logged in and located search box:", el_search)

**Part 2 - Set up list(s) of chat groups**

In [22]:
# list_chat_groups_test = [
#     'TestGroup01',
#     'TestGroup02',
#     'TestGroup03',
#     'TestGroup04',
#     'TestGroup05',
#     'TestGroup06',
#     'TestGroup07',
#     'TestGroup08',
#     'TestGroup09',
#     'TestGroup10'
# ]
file_path = 'C:\\Users\\Administrator\\Documents\\WhatsApp_Invite_Links.json'

with open(file_path, 'r') as file:
    list_chat_group_links = json.load(file)


**Part 3 - Define the automation functions**

_click_modal_button_ : a helper function used by the other functions.

_add_contact_to_group(group_name, contact_to_add)_ : adds **contact_to_add** to **group_name**

_remove_contact_from_group(group_name, contact_to_remove)_ : removes **contact_to_remove** from **group_name**

_make_group_admin(group_name, contact_to_add)_ : sets **contact_to_add** as admin of **group_name**

_dismiss_as_group_admin(group_name, contact_to_remove)_ : dismisses **contact_to_remove** as admin of **group_name**

In [23]:
#define a helper function
def click_modal_button(button_text):    
    modal_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//div[@role='button']//div[text() = '%s']" % (button_text))))
    modal_button.click()                                                      

#define a function that adds contact_to_add to group_name
def add_contact_to_group(group_name, contact_to_add):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()
        
    #wait for it to load by detecting that the header changed with the new title
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
    
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()    
    
    #click on the Add Participant button
    el_add_participant = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[text() = 'Add participant']")))
    el_add_participant.click()    
    
    #click on the Search
    el_modal_popup = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']")))    
    el_modal_popup.find_element(By.XPATH, "//div[contains(@title, 'Search')]").send_keys(contact_to_add)
    
    #click on the Contact
    el_contact_to_add = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//span[@title='%s']" % (contact_to_add))))
    el_contact_to_add.click()    
    
    #check whether already added
    if len(el_modal_popup.find_elements(By.XPATH, "//div[text() = 'Already added to group']")) > 0:
        print(contact_to_add + ' was already an existing participant of ' + group_name)
        el_modal_popup.find_element(By.XPATH, "//header//button[@aria-label='Close']").click()
    else:    
        #click on the Green Check Mark
        el_green_check = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-animate-modal-body='true']//div[@role='button']//span[@data-testid='checkmark-medium']")))
        el_green_check.click()        

        #click on the Add Participant
        click_modal_button('Add participant')
        print(contact_to_add + ' added to ' + group_name)


def remove_contact_from_group(group_name, contact_to_remove):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    try:
        el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_remove))))
        
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()

        #click on the Make group admin button
        el_remove_from_group = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Remove']")))
        el_remove_from_group.click()

        #click on the green button
        click_modal_button('Remove')
        print(contact_to_add + ' removed from ' + group_name)
    except:
        print("Error occured while finding user in chat")
        
        
def make_group_admin(group_name, contact_to_add):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_add))))
    
    #check whether user is already admin by looking for the Group admin label
    has_group_admin_label = len(el_target_contact.find_elements(By.XPATH, "../../..//div[text() = 'Group admin']"))    
    if has_group_admin_label > 0:    
        print(contact_to_add + (' were' if contact_to_add == 'You' else ' was') + ' already an existing admin of ' + group_name)
    else:
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()
    
        #click on the Make group admin button
        el_make_group_admin = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Make group admin']")))
        el_make_group_admin.click()
    
        #click on the green button
        click_modal_button('Make group admin')
        print(contact_to_add + ' added as admin of ' + group_name)

def dismiss_as_group_admin(group_name, contact_to_remove):
    #find chat with the correct title
    el_target_chat = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='%s']" % (group_name))))
    el_target_chat.click()    
        
    #wait for it to load by detecting header changed
    el_header_title = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//header//span[@title='%s']" % (group_name))))
        
    #click on the menu button
    el_menu_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='main']//div[@data-testid='conversation-menu-button']")))
    el_menu_button.click()
    
    #click on the Group Info button
    el_group_info = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Group info']")))
    el_group_info.click()
    
    #wait until target user can be found
    el_target_contact = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//span[@title='%s']" % (contact_to_remove))))
    
    #Check whether user is already admin by looking for the Group admin label
    has_group_admin_label = len(el_target_contact.find_elements(By.XPATH, "../../..//div[text() = 'Group admin']"))    
    if has_group_admin_label < 1:    
        print(contact_to_remove + (' were' if contact_to_remove == 'You' else ' was') + ' not an existing admin of ' + group_name)
    else:
        #Need to Hover over Contact
        #Ref: https://www.roelpeters.be/mouseover-in-selenium-hover/            
        ActionChains(driver).move_to_element(el_target_contact).perform() #hover over

        #Wait for dropdown arrow to appear, then click it.
        #Ref: https://stackoverflow.com/questions/27934945/selenium-move-to-element-does-not-always-mouse-hover
        el_dropdown_arrow = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='drawer-right']//div[@role='gridcell']//button[@aria-label='Open the chat context menu']")))
        el_dropdown_arrow.click()
    
        #click on the Make group admin button
        el_make_group_admin = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='app']//li//div[@aria-label='Dismiss as admin']")))
        el_make_group_admin.click()
            
        print(contact_to_remove + ' removed as admin of ' + group_name)   


def join_whatsapp_group(driver, invite_link):
    # Open the webpage of the invite link
    

    code = invite_link.split('/')[-1]
    whatsapp_web_url = f'https://web.whatsapp.com/accept?code={code}'
    logging.info(f"Navigating to: {whatsapp_web_url}")
    
    # Use execute_script to open a new tab with the whatsapp web url
    driver.execute_script("window.open(arguments[0]);", whatsapp_web_url)
    
    # Switch to the new tab to interact with it
    driver.switch_to.window(driver.window_handles[-1])
    
    try:
        # Wait for the 'Join Chat' button to be clickable
        wait = WebDriverWait(driver, 20) # Adjust the timeout to your needs
        join_chat_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/span[2]/div/div/div/div/div/div/div[2]/div/button[2]'))
        )
        
        # Click the 'Join Chat' button
        join_chat_button.click()
        # click_modal_button('Join Chat')

        logging.info(f"Joined WhatsApp group: {invite_link}")
        
         # Pause for 10 seconds at the end of the try block
        time.sleep(10)

    except NoSuchElementException as e:
        logging.error(f"Element not found: {e}")
    except TimeoutException as e:
        logging.error(f"Timeout while waiting for element: {e}")
    except WebDriverException as e:
        logging.error(f"WebDriver exception: {e}")

    finally:
    # Cleanup actions, if any
        if len(driver.window_handles) > 1:
            driver.close()
            # Switch back to the first tab if there are multiple tabs open
            driver.switch_to.window(driver.window_handles[0])

        
        # Switch back to the first tab if there are multiple tabs open
        if len(driver.window_handles) > 0:
            driver.switch_to.window(driver.window_handles[0])
        


In [ ]:
start_time = datetime.now()
size_of_list = len(list_chat_group_links)
counter = 0
#loop through a provided list of chat groups to perform an action
#REPLACE list_chat_groups_test with your chosen list name defined in Part 2
# for chat_name in list_chat_groups_test:
        
#     el_search.clear()
#     el_search.send_keys(chat_name)
    
#     try:

#         ##########################################################
#         ## COMMENT OUT the #1, #2, #3 or #4 BLOCKS as necessary ##
#         ##########################################################
        
#         #1 Add Contact to Group
#         print('Attempting to add to', chat_name, ":")
#         add_contact_to_group(chat_name, 'INSERT USER NAME HERE')

#         #2 Remove Contact from Group
#         print('Attempting to remove from group ', chat_name, ":")
#         remove_contact_from_group(chat_name, 'INSERT USER NAME HERE')      
        
#         #3 Set Contact as Group Admin
#         print('Attempting to make group admin of ', chat_name, ":")
#         make_group_admin(chat_name, 'INSERT USER NAME HERE')

#         #4 Dismiss Contact as Group Admin
#         print('Attempting to dismiss as admin of ', chat_name, ":")
#         dismiss_as_group_admin(chat_name, 'INSERT USER NAME HERE')
    
#         #4 Join Whatsapp Group
#         
#         dismiss_as_group_admin(chat_name, 'INSERT USER NAME HERE')
        
    # except Exception as exception:
    #     print("Exception: {}".format(type(exception).__name__))
    #     print("Exception message: {}".format(exception))

for chat_link in list_chat_group_links:

    try:
        
        logging.info(f'Trying to join: {chat_link}')
        join_whatsapp_group(driver, chat_link)
        current_time = datetime.now()
        print('Duration: {}'.format(current_time - start_time))
        
    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))

    finally:
        counter += 1
        percentage_done = (counter / size_of_list ) * 100
        rounded_percentage = round(percentage_done, 1)
        print(f"{rounded_percentage}% done")
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

2024-03-14 14:33:24,652 - INFO - Trying to join: https://chat.whatsapp.com/ECaPUNtaG9b8c1SlKlvJ6s
2024-03-14 14:33:24,654 - INFO - Navigating to: https://web.whatsapp.com/accept?code=ECaPUNtaG9b8c1SlKlvJ6s
2024-03-14 14:33:45,076 - ERROR - Timeout while waiting for element: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-03-14 14:33:45,217 - INFO - Trying to join: https://chat.whatsapp.com/G4c6HM2oif8K80yGtLT85U
2024-03-14 14:33:45,217 - INFO - Navigating to: https://web.whatsapp.com/accept?code=G4c6HM2oif8K80yGtLT85U


Duration: 0:00:20.564569
0.0% done


2024-03-14 14:34:05,498 - ERROR - Timeout while waiting for element: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-03-14 14:34:05,645 - INFO - Trying to join: https://chat.whatsapp.com/FqjQ80Zi2K5GpQuXO35YFZ
2024-03-14 14:34:05,645 - INFO - Navigating to: https://web.whatsapp.com/accept?code=FqjQ80Zi2K5GpQuXO35YFZ


Duration: 0:00:40.993178
0.1% done


2024-03-14 14:34:26,214 - ERROR - Timeout while waiting for element: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-03-14 14:34:26,350 - INFO - Trying to join: https://chat.whatsapp.com/BAvPi7Ple0qLXRX06MGf4N
2024-03-14 14:34:26,350 - INFO - Navigating to: https://web.whatsapp.com/accept?code=BAvPi7Ple0qLXRX06MGf4N


...

Duration: 7:34:20.187135
37.3% done


2024-03-14 22:08:05,870 - ERROR - Timeout while waiting for element: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-03-14 22:08:07,881 - INFO - Trying to join: https://chat.whatsapp.com/CDUkeRnRQY5IkemUXsNFuC
2024-03-14 22:08:07,881 - INFO - Navigating to: https://web.whatsapp.com/accept?code=CDUkeRnRQY5IkemUXsNFuC


Duration: 7:34:43.228519
37.4% done


2024-03-14 22:08:29,099 - ERROR - Timeout while waiting for element: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-03-14 22:08:31,200 - INFO - Trying to join: https://chat.whatsapp.com/F7cHMGvc8Kw7YWdvW5Jqol
2024-03-14 22:08:31,427 - INFO - Navigating to: https://web.whatsapp.com/accept?code=F7cHMGvc8Kw7YWdvW5Jqol


Duration: 7:35:06.548156
37.4% done
